# Práctica 3: Representación del conocimiento

__Fecha de entrega: 8 de mayo de 2022__

El objetivo de esta práctica es aplicar los conceptos teóricos vistos en clase en el módulo de Representación de conocimiento. La práctica consta de un único notebook que se entregará en la tarea de entrega habilitada en el Campus  Virtual.

Cada consulta debe contener breves comentarios que expliquen cada tripleta. __No se valorarán consultas sin explicaciones__.

__Número de grupo: 10__

__Nombres de los estudiantes: Adrián Sanjuan Espejo, Rubén Gómez Blanco y Adrián Pérez Peinador__

## Consultas SPARQL sobre Wikidata.

En esta práctica vamos a usar el [punto de acceso SPARQL](https://query.wikidata.org/) de Wikidata para contestar las preguntas que se formulan a continuación. Cada pregunta debe ser respondida realizando una única consulta SPARQL. Para cada una de las entidades recuperadas se mostrará __tanto su identificador como su etiqueta__ (nombre de la entidad en lenguaje natural). 

Para cada una de las preguntas debes mostrar tanto la consulta como la respuesta obtenida. Si lo consideras necesario, puedes añadir celdas adicionales en formato _Markdown_ para explicar decisiones que hayas tomado al crear la consulta o cualquier otro dato que consideres interesante.
 
Para resolver estas consultas puedes usar __dos recursos que te recomendamos consultar__ son:

- [Este tutorial de SPARQL](https://www.wikidata.org/wiki/Wikidata:SPARQL_tutorial).
- [Esta recopilación de ejemplos](https://www.wikidata.org/wiki/Wikidata:SPARQL_query_service/queries/examples)

In [39]:
import sys

!{sys.executable} -m pip install wdsparql

In [40]:
%load_ext autoreload
%load_ext wdsparql
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
The wdsparql extension is already loaded. To reload it, use:
  %reload_ext wdsparql


### Ejemplo

Recuperar todas las instancias directas de la clase [Cabra (Q2934)](https://www.wikidata.org/wiki/Q2934) que aparecen en la base de conocimiento.

In [41]:
%%wdsparql
SELECT ?item ?itemLabel 
WHERE 
{
  ?item wdt:P31 wd:Q2934.  # instancias directas de la clase Cabra
  SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
}

,item,itemLabel
0,http://www.wikidata.org/entity/Q151345,Billygoat Hennes
1,http://www.wikidata.org/entity/Q3569037,William Windsor
2,http://www.wikidata.org/entity/Q23003932,His Whiskers
3,http://www.wikidata.org/entity/Q24287064,Taffy
4,http://www.wikidata.org/entity/Q41239734,Lance Corporal Shenkin III
5,http://www.wikidata.org/entity/Q41240892,Lance Corporal Shenkin II
6,http://www.wikidata.org/entity/Q41241416,Lance Corporal Shenkin I


### Consulta 1

[Isaac Asimov (Q34981)](https://www.wikidata.org/wiki/Q34981) fue un escritor y profesor de bioquímica en la Universidad de Boston conocido por ser un prolífico autor de obras de ciencia ficción, historia y divulgación científica. 

Vamos a comenzar por averiguar su fecha y lugar de nacimiento (localidad y país al que pertenece la localidad en la actualidad).

In [53]:
%%wdsparql
SELECT  ?nacimiento ?nacimientoLabel ?lugar ?lugarLabel ?pais ?paisLabel
WHERE 
{
  wd:Q34981 wdt:P569 ?nacimiento. # Isaac Asimov -> Fecha de nacimiento
  wd:Q34981 wdt:P19 ?lugar.       # Isaac Asimov -> Lugar de nacimiento
  ?lugar wdt:P17 ?pais            # ?lugar -> pais -> ?pais
  SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
  
}

,nacimiento,nacimientoLabel,lugar,lugarLabel,pais,paisLabel
0,1920-01-02T00:00:00Z,1920-01-02T00:00:00Z,http://www.wikidata.org/entity/Q1010621,Petrovichi,http://www.wikidata.org/entity/Q159,Russia


### Consulta 2

A continuación vamos a averiguar todas las distintas profesiones (ocupaciones) que se le reconocen en la base de conocimiento. Queremos obtener los resultados ordenados alfabéticamente por el nombre de la profesión.

In [43]:
%%wdsparql
SELECT ?profesion ?profesionLabel
WHERE 
{
  wd:Q34981 wdt:P106 ?profesion. #Isaac Asimov -> ocupación

  SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
  
}
ORDER BY ?profesionLabel # Ordenar alfabéticamente por profesión

,profesion,profesionLabel
0,http://www.wikidata.org/entity/Q18814623,autobiographer
1,http://www.wikidata.org/entity/Q2919046,biochemist
2,http://www.wikidata.org/entity/Q1930187,journalist
3,http://www.wikidata.org/entity/Q15980158,non-fiction writer
4,http://www.wikidata.org/entity/Q6625963,novelist
5,http://www.wikidata.org/entity/Q12144794,prosaist
6,http://www.wikidata.org/entity/Q18844224,science fiction writer
7,http://www.wikidata.org/entity/Q3745071,science writer
8,http://www.wikidata.org/entity/Q901,scientist
9,http://www.wikidata.org/entity/Q28389,screenwriter


### Consulta 3

De todas esas profesiones, ¿cuáles corresponden con subtipos de [Escritor (Q36180)](https://www.wikidata.org/wiki/Q36180)? Ten en cuenta que la jerarquía de tipos de escritores puede ser muy sofisticada.

Puede que [esto](https://www.wikidata.org/wiki/Wikidata:SPARQL_tutorial#Property_paths) te resulte útil.

In [44]:
%%wdsparql
SELECT ?profesion ?profesionLabel
WHERE 
{
  wd:Q34981 wdt:P106 ?profesion. # Isaac Asimov -> ocupación
  ?profesion wdt:P279+ wd:Q36180. # Subclases de escritor y subclases de susbclases de escritor recurrentemente
  SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
}
ORDER BY ?profesionLabel # Ordenar alfabéticamente por profesión

,profesion,profesionLabel
0,http://www.wikidata.org/entity/Q18814623,autobiographer
1,http://www.wikidata.org/entity/Q2919046,biochemist
2,http://www.wikidata.org/entity/Q1930187,journalist
3,http://www.wikidata.org/entity/Q15980158,non-fiction writer
4,http://www.wikidata.org/entity/Q6625963,novelist
5,http://www.wikidata.org/entity/Q12144794,prosaist
6,http://www.wikidata.org/entity/Q18844224,science fiction writer
7,http://www.wikidata.org/entity/Q3745071,science writer
8,http://www.wikidata.org/entity/Q901,scientist
9,http://www.wikidata.org/entity/Q28389,screenwriter


### Consulta 4

Asimov se casó más de una vez. Para cada uno de los matrimonios queremos conocer el nombre de la esposa y las fechas de inicio y finalización. Los resultados deben aparecer ordenados cronológicamente.

Para resolver esta consulta necesitarás acceder a los cualificadores de nodos sentencia y necesitarás entender los prefijos que usa Wikidata. Puede que [esto](https://www.wikidata.org/wiki/Wikidata:SPARQL_tutorial#Qualifiers) te resulte útil.

In [45]:
%%wdsparql
SELECT ?nombreesposa ?nombreesposaLabel ?fechaini ?fechainiLabel ?fechafin ?fechafinLabel
WHERE 
{
  wd:Q34981 p:P26 ?esposa. #Isaac Asimov -> Esposas -> ?esposa
  ?esposa ps:P26 ?nombreesposa. #esposa -> nombre_de_statement (nombre de la esposa)
  ?esposa pq:P580 ?fechaini. #esposa -> Fecha inicio
  ?esposa pq:P582 ?fechafin. #esposa -> Fecha fin
  SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
  
}
ORDER BY ?fechaini #Ordenados por fecha inicio

,nombreesposa,nombreesposaLabel,fechaini,fechainiLabel,fechafin,fechafinLabel
0,http://www.wikidata.org/entity/Q107693884,Gertrude Asimov,1942-07-26T00:00:00Z,1942-07-26T00:00:00Z,1973-01-01T00:00:00Z,1973-01-01T00:00:00Z
1,http://www.wikidata.org/entity/Q517435,Janet Asimov,1973-01-01T00:00:00Z,1973-01-01T00:00:00Z,1992-01-01T00:00:00Z,1992-01-01T00:00:00Z


### Consulta 5

Asimov recibió muchos premios a lo largo de su carrera. Queremos obtener la lista de premios y, para cada uno de ellos, la fecha y la obra por la que fue premiado (si están disponibles). Los resultados se deben mostrar ordenador por fecha.

Puede que [esto](https://www.wikidata.org/wiki/Wikidata:SPARQL_tutorial#OPTIONAL) te resulte útil.

In [46]:
%%wdsparql
SELECT ?premio ?premioLabel ?fecha ?fechaLabel ?obra ?obraLabel
WHERE 
{
  wd:Q34981 p:P166 ?p. #Isaac Asimov -> Premios -> ?premio
  ?p ps:P166 ?premio. #p -> nombre_de_p (nombre del premio)
  ?p pq:P585 ?fecha. #p -> fecha
  OPTIONAL{?p pq:P1686 ?obra. } #p -> obra (si la hay)
  SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
  
}
ORDER BY ?fecha #Ordenados por fecha

,premio,premioLabel,fecha,fechaLabel,obra,obraLabel
0,http://www.wikidata.org/entity/Q188914,Hugo Award,1963-01-01T00:00:00Z,1963-01-01T00:00:00Z,None,None
1,http://www.wikidata.org/entity/Q6143927,James T. Grady-James H. Stack Award for Interp...,1965-01-01T00:00:00Z,1965-01-01T00:00:00Z,None,None
2,http://www.wikidata.org/entity/Q188914,Hugo Award,1966-01-01T00:00:00Z,1966-01-01T00:00:00Z,http://www.wikidata.org/entity/Q20899148,Foundation Trilogy
3,http://www.wikidata.org/entity/Q987744,Edward E. Smith Memorial Award,1967-01-01T00:00:00Z,1967-01-01T00:00:00Z,None,None
4,http://www.wikidata.org/entity/Q266012,Nebula Award for Best Novel,1972-01-01T00:00:00Z,1972-01-01T00:00:00Z,http://www.wikidata.org/entity/Q389750,The Gods Themselves
5,http://www.wikidata.org/entity/Q255032,Hugo Award for Best Novel,1973-01-01T00:00:00Z,1973-01-01T00:00:00Z,http://www.wikidata.org/entity/Q389750,The Gods Themselves
6,http://www.wikidata.org/entity/Q595998,Locus Award for Best Novel,1973-01-01T00:00:00Z,1973-01-01T00:00:00Z,http://www.wikidata.org/entity/Q389750,The Gods Themselves
7,http://www.wikidata.org/entity/Q3174253,Klumpke-Roberts Award,1975-01-01T00:00:00Z,1975-01-01T00:00:00Z,None,None
8,http://www.wikidata.org/entity/Q936683,Nebula Award for Best Novelette,1976-01-01T00:00:00Z,1976-01-01T00:00:00Z,http://www.wikidata.org/entity/Q468132,The Bicentennial Man
9,http://www.wikidata.org/entity/Q1056265,Hugo Award for Best Novelette,1977-01-01T00:00:00Z,1977-01-01T00:00:00Z,http://www.wikidata.org/entity/Q468132,The Bicentennial Man


### Consulta 6

Ahora queremos conocer todas las obras escritas por Asimov del tipo [obras literarias de ciencia ficción (Q3238422)](https://www.wikidata.org/wiki/Q3238422) y cualquiera de sus subgéneros. Los resultados se deben mostrar ordenados por nombre.

No pasa nada si en el resultado aparecen mezclados novelas, relatos, colecciones...

In [47]:
%%wdsparql
SELECT ?item ?itemLabel ?genero ?generoLabel
WHERE {
  ?item wdt:P50 wd:Q34981.# ?item -> autor -> Isaac Asimov
  OPTIONAL {?item wdt:P136 ?genero.} # ?item -> genero -> ?genero (si lo hay)
  OPTIONAL {?genero pq:P279 wd:Q3238422.} # ?genero -> subclase de -> literatura de ciencia ficcion (si tiene genero)
  SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }
}
ORDER BY ?itemLabel

,item,itemLabel,genero,generoLabel
0,http://www.wikidata.org/entity/Q104865083,Q104865083,None,None
1,http://www.wikidata.org/entity/Q104868766,Q104868766,http://www.wikidata.org/entity/Q24925,science fiction
2,http://www.wikidata.org/entity/Q104868864,Q104868864,http://www.wikidata.org/entity/Q24925,science fiction
3,http://www.wikidata.org/entity/Q104868972,Q104868972,http://www.wikidata.org/entity/Q24925,science fiction
4,http://www.wikidata.org/entity/Q104869051,Q104869051,http://www.wikidata.org/entity/Q24925,science fiction
5,http://www.wikidata.org/entity/Q12090866,Q12090866,None,None
6,http://www.wikidata.org/entity/Q16069681,Q16069681,http://www.wikidata.org/entity/Q24925,science fiction
7,http://www.wikidata.org/entity/Q16652396,Q16652396,None,None
8,http://www.wikidata.org/entity/Q17174405,Q17174405,None,None
9,http://www.wikidata.org/entity/Q25492175,Q25492175,http://www.wikidata.org/entity/Q24925,science fiction


O si queremos que no se repitan obras con sus diferentes subgéneros:

In [48]:
%%wdsparql
SELECT ?item ?itemLabel
WHERE {
  ?item wdt:P50 wd:Q34981.
  OPTIONAL {?item wdt:P136 ?genero.} # ?item -> genero -> ?genero (si lo hay)
  OPTIONAL {?genero pq:P279 wd:Q3238422.} # ?genero -> subclase de -> literatura de ciencia ficcion (si tiene genero)
  SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }
}
GROUP BY ?item ?itemLabel
ORDER BY ?itemLabel

,item,itemLabel
0,http://www.wikidata.org/entity/Q104865083,Q104865083
1,http://www.wikidata.org/entity/Q104868766,Q104868766
2,http://www.wikidata.org/entity/Q104868864,Q104868864
3,http://www.wikidata.org/entity/Q104868972,Q104868972
4,http://www.wikidata.org/entity/Q104869051,Q104869051
5,http://www.wikidata.org/entity/Q12090866,Q12090866
6,http://www.wikidata.org/entity/Q16069681,Q16069681
7,http://www.wikidata.org/entity/Q16652396,Q16652396
8,http://www.wikidata.org/entity/Q17174405,Q17174405
9,http://www.wikidata.org/entity/Q25492175,Q25492175


### Consulta 7

Vamos a volver a recuperar las mismas obras de la consulta anterior pero mostrando también la fecha de publicación de cada una. Las obras deben aparece aunque no tengan fecha de publicación asociada. En caso de que una obra tenga varias fechas de publicación, se mostrará sólo la más antigua. Los resultados deben aparecer en orden cronológico.

Para calcular la fecha correcta tendrás que agrupar las respuestas por obra y aplicar una función de agregación sobre las fechas de publicación. Puede que [esto](https://www.wikidata.org/wiki/Wikidata:SPARQL_tutorial#Grouping) te resulte útil.

In [49]:
%%wdsparql
SELECT ?item ?itemLabel ?genero ?generoLabel (MIN(?fecha) as ?fecha_publ) # ?fecha_publ es el minimo de las ?fecha en cada grupo
WHERE {
  ?item wdt:P50 wd:Q34981.  # ?item -> autor -> Isaac Asimov
  OPTIONAL {?item wdt:P136 ?genero.} # ?item -> genero -> ?genero (si lo hay)
  OPTIONAL {?genero pq:P279 wd:Q3238422.} # ?genero -> subclase de -> literatura de ciencia ficcion (si tiene genero)
  OPTIONAL {?item wdt:P577 ?fecha.}  # ?item -> fecha de publicacion -> ?fecha
  SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }
}
GROUP BY ?item ?itemLabel ?genero ?generoLabel  # Agrupamos por ?item y ?genero iguales
ORDER BY ?itemLabel

,item,itemLabel,genero,generoLabel,fecha_publ
0,http://www.wikidata.org/entity/Q104865083,Q104865083,None,None,1974-06-18T00:00:00Z
1,http://www.wikidata.org/entity/Q104868766,Q104868766,http://www.wikidata.org/entity/Q24925,science fiction,1973-03-16T00:00:00Z
2,http://www.wikidata.org/entity/Q104868864,Q104868864,http://www.wikidata.org/entity/Q24925,science fiction,1973-04-13T00:00:00Z
3,http://www.wikidata.org/entity/Q104868972,Q104868972,http://www.wikidata.org/entity/Q24925,science fiction,1973-05-11T00:00:00Z
4,http://www.wikidata.org/entity/Q104869051,Q104869051,http://www.wikidata.org/entity/Q24925,science fiction,1974-10-08T00:00:00Z
5,http://www.wikidata.org/entity/Q12090866,Q12090866,None,None,None
6,http://www.wikidata.org/entity/Q16069681,Q16069681,http://www.wikidata.org/entity/Q24925,science fiction,None
7,http://www.wikidata.org/entity/Q16652396,Q16652396,None,None,1972-01-01T00:00:00Z
8,http://www.wikidata.org/entity/Q17174405,Q17174405,None,None,1983-01-01T00:00:00Z
9,http://www.wikidata.org/entity/Q25492175,Q25492175,http://www.wikidata.org/entity/Q24925,science fiction,1969-01-01T00:00:00Z


### Consulta 8

Asimov es sin duda un autor prolífico. ¿Cuántas obras escribió a lo largo de su vida? Vamos a considerar sólo instancias directas de [literary work (Q7725634)](https://www.wikidata.org/wiki/Q7725634).

In [50]:
%%wdsparql
SELECT ?tipo ?tipoLabel (COUNT(?item) as ?numeroDeObras) #contamos el número de elementos en cada grupo
WHERE {
  ?item wdt:P50 wd:Q34981. # ?item -> autor -> Isaac Asimov
  OPTIONAL {?item wdt:P31 ?tipo.} # ?item -> instancia de -> ?tipo
  SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }
}
GROUP BY ?tipo ?tipoLabel  #Agrupamos por ?tipo
HAVING (?tipo = wd:Q7725634) #Restringimos el valor de ?tipo a obra literaria

,tipo,tipoLabel,numeroDeObras
0,http://www.wikidata.org/entity/Q7725634,literary work,272


### Consulta 9

De todas las obras que escribió a lo largo de su vida, queremos recuperar los nombres y fechas de publicación de aquellas escritas entre 1970 y 1980. Si alguna obra tiene varias fechas de publicación, se mostrará si alguna de ellas está dentro de ese periodo.

Puede que [esto](https://www.wikidata.org/wiki/Wikidata:SPARQL_tutorial#FILTER) te resulte útil.

In [51]:
%%wdsparql
SELECT ?item ?itemLabel ?fecha
WHERE {
  ?item wdt:P50 wd:Q34981. # ?item -> autor -> Isaac Asimov
  ?item wdt:P577 ?fecha.  # ?item -> fecha de pubicacion -> ?fecha
  FILTER (YEAR(?fecha) > 1970 && YEAR(?fecha) < 1980) # Nos quedamos con los resultados en los que 1970 < ?fecha < 1980
  SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }
}
ORDER BY ?fecha

,item,itemLabel,fecha
0,http://www.wikidata.org/entity/Q16932955,The Stars in their Courses,1971-01-01T00:00:00Z
1,http://www.wikidata.org/entity/Q16652396,Q16652396,1972-01-01T00:00:00Z
2,http://www.wikidata.org/entity/Q55172458,Q55172458,1972-01-01T00:00:00Z
3,http://www.wikidata.org/entity/Q98829663,asimov's guide to science,1972-01-01T00:00:00Z
4,http://www.wikidata.org/entity/Q6404541,Kid Stuff,1972-01-01T00:00:00Z
5,http://www.wikidata.org/entity/Q7677898,Take a Match,1972-01-01T00:00:00Z
6,http://www.wikidata.org/entity/Q7738024,The Greatest Asset,1972-01-01T00:00:00Z
7,http://www.wikidata.org/entity/Q7746627,The Left Hand of the Electron,1972-01-01T00:00:00Z
8,http://www.wikidata.org/entity/Q3345921,Q3345921,1972-01-01T00:00:00Z
9,http://www.wikidata.org/entity/Q389750,The Gods Themselves,1972-01-01T00:00:00Z


### Consulta 10

¿Qué otros escritores de ciencia ficción estudiaron en algún centro donde también estudió Asimov? Para cada uno de ellos muestra su nombre y fechas de nacimiento y defunción (si están disponibles) y el centro donde estudió. Muestra los resultados ordenados alfabéticamente por centro y luego por autor.

In [52]:
%%wdsparql
SELECT ?writer ?writerLabel ?school ?schoolLabel (MIN(?fechanac) as ?fechaNac) (MIN(?fechadef) as ?fechaDef)
WHERE {
  wd:Q34981 wdt:P69 ?school.            # Isaac Asimov -> educado en -> ?school
  ?writer wdt:P69 ?school.              # ?writer -> educado en -> ?school
  ?book wdt:P50 ?writer.                # ?book -> autor -> ?writer
  ?book wdt:P136 ?genero.               # ?book -> género -> ?genero
  ?genero wdt:P279* wd:Q3238422.        # ?genero -> (subclase de)* -> lieratura de ciencia ficcion
  OPTIONAL{?writer wdt:P569 ?fechanac.} # ?writer -> fecha de nacimiento -> ?fechanac (si la hay)
  OPTIONAL{?writer wdt:P570 ?fechadef.} # ?writer -> fecha de fallecimiento -> ?fechanac (si la hay)
  FILTER(?writer != wd:Q34981).         # quitamos al propio Asimov
  SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }
}
GROUP BY ?writer ?writerLabel ?school ?schoolLabel # Agrupamos por autor y centro
ORDER BY ?schoolLabel ?writerLabel      # Ordenamos por centro y luego por autor

,writer,writerLabel,school,schoolLabel,fechaNac,fechaDef
0,http://www.wikidata.org/entity/Q547414,Algis Budrys,http://www.wikidata.org/entity/Q49088,Columbia University,1931-01-09T00:00:00Z,2008-06-09T00:00:00Z
1,http://www.wikidata.org/entity/Q191716,Gene Roddenberry,http://www.wikidata.org/entity/Q49088,Columbia University,1921-08-19T00:00:00Z,1991-10-24T00:00:00Z
2,http://www.wikidata.org/entity/Q248867,James Blish,http://www.wikidata.org/entity/Q49088,Columbia University,1921-05-23T00:00:00Z,1975-07-30T00:00:00Z
3,http://www.wikidata.org/entity/Q242643,Naomi Novik,http://www.wikidata.org/entity/Q49088,Columbia University,1973-04-30T00:00:00Z,None
4,http://www.wikidata.org/entity/Q214642,Paul Auster,http://www.wikidata.org/entity/Q49088,Columbia University,1947-02-03T00:00:00Z,None
5,http://www.wikidata.org/entity/Q314553,Robert Silverberg,http://www.wikidata.org/entity/Q49088,Columbia University,1935-01-15T00:00:00Z,None
6,http://www.wikidata.org/entity/Q295406,Roger Zelazny,http://www.wikidata.org/entity/Q49088,Columbia University,1937-05-13T00:00:00Z,1995-06-14T00:00:00Z
7,http://www.wikidata.org/entity/Q181659,Ursula K. Le Guin,http://www.wikidata.org/entity/Q49088,Columbia University,1929-10-21T00:00:00Z,2018-01-22T00:00:00Z
8,http://www.wikidata.org/entity/Q16214455,Zoltan Istvan,http://www.wikidata.org/entity/Q49088,Columbia University,1973-03-30T00:00:00Z,None


__Fecha de las consultas: 08 de mayo de 2022__